## Gemini 기본 처리

### 라이브러리 설치

In [1]:
%pip install --upgrade --quiet google-genai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.5/218.5 kB 4.2 MB/s eta 0:00:00


### GCP 환경설정 및 로그인

In [2]:
import os

PROJECT_ID = "ai-hangsik" #@param {type:"string"}
REGION = "us-central1" #@param {type:"string"}
USE_VERTEX_AI = True #@param {type:"boolean"}
MODEL = "gemini-2.5-flash" #@param {type:"string"}

In [ ]:
!gcloud auth application-default login
!gcloud auth application-default set-quota-project {PROJECT_ID}

### Gemini 실행

In [4]:
import base64
from IPython.display import Image, display, Markdown

from google import genai
from google.genai import types
from google.genai.types import HttpOptions

client = genai.Client(
    vertexai=USE_VERTEX_AI,
    project=PROJECT_ID,
    location=REGION,)

#### Text 입력

In [6]:
from google import genai
from google.genai.types import HttpOptions
from IPython.display import Image, display, Markdown


response = client.models.generate_content(
    model=MODEL,

    contents=[
      types.Content(
        role="user",
        parts=[
          types.Part.from_text(text="""트랜스포머 아키텍처에서 인코더란? """)
        ]
      )
    ],
)
display(Markdown(response.text))

트랜스포머(Transformer) 아키텍처에서 **인코더(Encoder)**는 입력 시퀀스(예: 문장)를 처리하여 의미 있는 **컨텍스트 벡터(Context Vector)** 또는 **표현(Representation)**으로 변환하는 역할을 하는 부분입니다.

쉽게 말해, 인코더는 입력된 정보를 '이해'하고 '핵심을 추출'하는 두뇌와 같은 역할을 합니다.

### 인코더의 주요 기능 및 구성

1.  **핵심 기능: 입력 시퀀스 이해 및 인코딩**
    *   주어진 입력 텍스트(예: 한국어 문장 "나는 오늘 아침에 사과를 먹었다.")의 각 단어 또는 토큰 간의 관계와 문맥을 파악하여, 그 정보를 압축된 고차원 벡터 공간의 형태로 인코딩합니다.
    *   이 인코딩된 정보는 나중에 디코더(Decoder)가 새로운 시퀀스(예: 영어 번역 문장 "I ate an apple this morning.")를 생성하는 데 필요한 기반이 됩니다.

2.  **구조: 여러 개의 동일한 인코더 레이어(Encoder Layer)로 구성**
    *   인코더는 보통 **N개의 동일한 인코더 레이어를 쌓아 올린 형태**로 구성됩니다 (일반적으로 N=6).
    *   각 인코더 레이어는 다음 두 가지 주요 서브 레이어(Sub-layer)로 이루어져 있습니다:

    a.  **멀티헤드 셀프 어텐션(Multi-Head Self-Attention) 레이어:**
        *   이 부분이 인코더의 핵심입니다. 입력 시퀀스 내의 **모든 단어들이 서로에게 얼마나 연관되어 있는지**를 계산하여 문맥적 의미를 파악합니다.
        *   예를 들어, "The animal didn't cross the street because **it** was too tired." 문장에서 'it'이 'animal'을 가리킨다는 것을 스스로 학습하는 능력을 가집니다.
        *   '멀티헤드'는 하나의 어텐션이 아니라 여러 개의 '헤드'를 사용하여 다양한 관점에서 단어 간의 관계를 동시에 학습하므로 더 풍부하고 다면적인 문맥 정보를 얻을 수 있습니다.

    b.  **포지션-와이즈 피드 포워드 네트워크(Position-wise Feed-Forward Network) 레이어:**
        *   셀프 어텐션 레이어를 거쳐 나온 각 단어의 벡터 표현에 비선형 변환을 적용합니다.
        *   이는 모델의 표현력을 높이고, 각 단어의 벡터를 더 정교하게 다듬는 역할을 합니다. 각 포지션에 독립적으로 적용되지만, 레이어 내의 모든 포지션에 동일한 가중치가 적용됩니다.

3.  **추가적인 구조적 특징:**
    *   **잔차 연결(Residual Connections):** 각 서브 레이어의 입력과 출력을 더하여 정보 손실을 방지하고 깊은 네트워크의 학습을 용이하게 합니다.
    *   **레이어 정규화(Layer Normalization):** 각 서브 레이어의 출력에 적용되어 학습 안정성을 높이고 수렴 속도를 빠르게 합니다.

4.  **입력 및 출력:**
    *   **입력:** 인코더의 첫 번째 레이어에는 **단어 임베딩(Word Embeddings)**과 **위치 임베딩(Positional Embeddings)**이 더해져서 들어갑니다. 단어 임베딩은 단어의 의미를 벡터로 표현하고, 위치 임베딩은 단어의 순서 정보를 제공합니다.
    *   **출력:** 최종 인코더 레이어에서 출력되는 것은 입력 시퀀스의 각 단어에 대한 **문맥 정보가 풍부하게 담긴 벡터들의 시퀀스**입니다. 이 인코딩된 출력은 디코더의 특정 어텐션 레이어(Encoder-Decoder Attention)로 전달되어, 디코더가 출력을 생성할 때 입력 시퀀스의 어떤 부분에 집중해야 할지 알려주는 역할을 합니다.

### 비유적 설명

트랜스포머 인코더는 마치 **'똑똑한 독서가'**와 같습니다. 주어진 글을 처음부터 끝까지 꼼꼼히 읽고, 각 단어의 의미와 함께 단어들 간의 복잡한 관계(문맥)를 완벽하게 파악하여, 그 내용을 잘 요약된 형태로 변환하는 역할을 수행합니다. 이렇게 잘 요약된 정보는 나중에 '작가'(디코더)가 새로운 글을 쓰는 데 참조됩니다.

요약하자면, 트랜스포머 인코더는 입력 시퀀스의 복잡한 문맥적 의존성을 파악하고 이를 고차원 벡터 표현으로 압축하는 역할을 하며, 트랜스포머의 성공에 핵심적인 기여를 합니다.

#### Image 입력

In [7]:
img_file_uri="https://storage.googleapis.com/cloud-samples-data/generative-ai/image/scones.jpg"
display(Image(url=img_file_uri, width=400))

In [8]:
from google import genai
from google.genai.types import HttpOptions, Part

response = client.models.generate_content(
    model=MODEL,
    contents=[
        Part.from_text(text="""이미지에 보여지는 내용을 설명해주세요"""),
        Part.from_uri(
            file_uri= img_file_uri,
            mime_type="image/jpeg",
        ),
    ],
)
display(Markdown(response.text))

이 이미지는 블루베리 스콘과 신선한 과일, 음료, 꽃 등이 어우러진 아늑하고 먹음직스러운 식탁 풍경을 담고 있습니다.

사진 중앙에는 여러 개의 둥근 블루베리 스콘 또는 비스킷이 하얀 유산지 위에 놓여 있습니다. 스콘은 겉면이 살짝 노릇하고 설탕이 뿌려져 있으며, 속에는 보랏빛 블루베리가 박혀 있어 달콤하고 상큼해 보입니다. 유산지 위에는 블루베리 즙이나 흔적이 군데군데 묻어 있어 자연스러운 느낌을 더합니다.

스콘 주변으로는 다음과 같은 요소들이 배치되어 있습니다:
*   **음료:** 왼쪽 상단에는 진한 색의 커피가 담긴 하얀 머그컵이 보입니다. 오른쪽 하단에는 우유나 크림이 들어간 듯한 연한 갈색 음료가 담긴 또 다른 하얀 잔이 놓여 있습니다.
*   **신선한 블루베리:** 사진 중앙 아래쪽에는 작은 갈색 그릇에 신선한 블루베리가 가득 담겨 있습니다. 스콘 주변 유산지 위에도 몇몇 블루베리가 흩어져 있습니다.
*   **스푼:** 블루베리 그릇 옆에는 "LET'S JAM"이라고 새겨진 은색 스푼이 놓여 있어, 잼과 함께 스콘을 즐기는 분위기를 연출합니다.
*   **꽃:** 사진의 오른편에는 여러 송이의 분홍색 모란(작약) 꽃이 길게 배치되어 있어 화사하고 우아한 분위기를 더합니다.
*   **민트 잎:** 스콘 근처에는 작은 초록색 민트 잎 하나가 놓여 있어 신선함을 강조합니다.

전체적인 배경은 어둡고 질감 있는 표면으로, 따뜻하고 소박하면서도 세련된 느낌을 줍니다. 이 이미지는 여유로운 아침 식사나 오후 티타임을 연상시키며, 신선한 재료와 정성이 느껴지는 디저트의 아름다움을 시각적으로 잘 표현하고 있습니다.

#### Youtube 분석

In [9]:
from google import genai
from google.genai.types import HttpOptions, Part

response = client.models.generate_content(
    model=MODEL,
    contents=[
        Part.from_uri(
            file_uri="https://www.youtube.com/watch?v=3KtWfp0UopM",
            mime_type="video/mp4",
        ),
        Part.from_text(text="""이 영상을 바탕으로 짧고 매력적인 블로그 게시물을 한국어로 작성해 보세요."""),
    ],
)

display(Markdown(response.text))

## 구글과 함께한 25년: 가장 많이 검색된 이야기들 ✨

25년 전, 구글과 함께 세상은 '검색'이라는 새로운 여정을 시작했습니다. 지난 25년간 구글은 전 세계 수십억 명의 사람들이 궁금해하는 모든 것을 찾아볼 수 있도록 도왔죠. 최근 공개된 구글 25주년 기념 "가장 많이 검색된(The Most Searched)" 비디오는 단순한 검색 기록을 넘어, 우리 시대의 관심사와 역사를 보여주는 감동적인 여정입니다.

영상은 인류의 위대한 첫걸음이었던 **달 착륙**부터 시작해, 가장 많이 검색된 **1980년대**의 문화, 그리고 **포켓몬, 해리포터, 심슨 가족** 같은 추억의 콘텐츠까지 다채로운 과거를 되짚습니다. **BTS**와 **블랙핑크** 같은 K-Pop 아이돌, **비욘세**의 압도적인 공연, **바비** 인형의 진화까지, 검색은 우리가 사랑하고 열광하는 문화 현상의 중심에 늘 함께했습니다.

스포츠 스타 **호날두**와 **르브론 제임스**, 크리켓 선수 **비라트 콜리**를 통해 최고들의 열정을 보여주고, 스파이더맨 같은 슈퍼히어로뿐만 아니라 소방관, 의료진, 재난 구조대원 같은 현실 영웅들의 숭고한 정신을 조명합니다. 핵융합과 같은 과학적 돌파구, 그리고 블랙 라이브스 매터처럼 중요한 사회 운동까지, 검색은 우리 사회의 발전과 인류의 노력을 기록해왔습니다.

특히 '도움이 필요한 사람 돕는 법'과 같은 검색어는 우리가 가진 선한 의지를 보여줍니다. 아이스버킷 챌린지 같은 바이럴 캠페인, 그리고 프라이드 퍼레이드처럼 다양성을 존중하는 움직임들은 검색이 단순한 정보 전달을 넘어, 공동체 의식을 형성하고 긍정적인 변화를 이끌어냈음을 증명하죠.

영상 마지막에 등장하는 "기준을 세우는 사람들, 그리고 그 기준을 더 높이 올릴 사람들"이라는 메시지는 미래 세대에 대한 희망과 기대를 담고 있습니다. 어린 아이들이 농구, 축구, 역도, 스케이트보드 등 다양한 분야에서 잠재력을 뽐내는 모습은 검색이 단순히 과거를 보여주는 것을 넘어, 미래를 향한 무한한 가능성을 열어준다는 것을 상기시킵니다.

지난 25년간 구글 검색은 우리 삶의 거울이자, 지식과 영감의 원천이었습니다. 과거를 기억하고, 현재를 이해하며, 더 나은 미래를 꿈꾸게 하는 힘. 이제 다음 25년을 향해, 계속해서 '검색하세요(Search on).'

구글의 25주년을 축하합니다! 🎉

## End of Document